# cx_Oracle 모듈
- 파이썬에서 오라클 DBMS와 연동는 다양한 함수를 제공하는 모듈
- Python DB API 2.0 표준을 따른다. 

## 참고
- [cx_Oracle 모듈 홈페이지](https://oracle.github.io/python-cx_Oracle/)
- [Python DB API](https://docs.python.org/ko/3/library/sqlite3.html)

## cx_Oracle 모듈 설치
- [설치 튜토리얼](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)
- 패키지 설치
```
conda install cx_oracle
pip install cx_Oracle --upgrade
```

## 코딩 절차
1. cx_Oracle 모듈 import

2. cx_Oracle모듈의 connect() 함수를 이용해 디비연결

3. Connection 객체의 cursor() 메소드를 이용해 Cursor객체 조회.
    - Cursor객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체

4. Cursor 의 execute() 메소드를 이용해 SQL문 전송

5. SELECT 의 경우 결과 조회.
    - 조회결과 한행씩 반환받아 사용
    - 조회결과 한번에 반환받아 사용

6. INSERT/DELETE/UPDATE의 경우  SQL 문 실행 후 Connection의 commit() 메소드를 이용해 Transaction 처리

7. Cursor, Connection 연결 닫기

### DB 연결
- connect() 함수를 이용하며 연결 후 Connection 객체를 받는다.
- 연결시 필요한 값
    - **host**
    - **port번호**
    - **SID(DB이름)**
    - **user name**
    - **password**

### Connection 주요 메소드
- commit(): 커밋
- rollback(): 롤백
- close(): 연결 닫기

In [2]:
# connect() 함수를 이용해 연결. 연결(접속) 성공하면 Connection 객체를 반환
# Connection 객체: 연결 정보를 가지고 있는 객체
import cx_Oracle
username = 'c##scott_join'
password = 'tiger'
url = "localhost:1521/XE"  #host:port/sid
conn = cx_Oracle.connect(username, password, url) #연결
print(type(conn))
conn.close()

<class 'cx_Oracle.Connection'>


In [5]:
# username/pwd@url
conn2 = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
print(type(conn2))
conn2.close()

<class 'cx_Oracle.Connection'>


#### DSN (Data Source Name) 을 이용한 연결
- DSN: DBMS를 연결하기 위한 정보를 모아 놓은 것. (서로 다른 DBMS의 연결정보를 통일하기 위해서 방식)
- makedsn(host, port번호, SID) 함수 이용해 생성한다.

In [7]:
username = "c##scott_join"
password = "tiger"
host = "localhost"
port = 1521
sid = "XE"
dsn = cx_Oracle.makedsn(host, port, sid)
print(dsn)
conn3 = cx_Oracle.connect(username, password, dsn)
print(type(conn3))
conn3.close()

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))
<class 'cx_Oracle.Connection'>


In [ ]:
with 

## Cursor 객체
- SQL문을 전송하고 select결과를 조회하는 메소드들을 제공한다.
- Connection객체의 cursor() 메소드로 받아온다.

### Cursor의 주요 메소드
#### SQL 실행 메소드
- execute(sql) : 하나의 sql 문 실행. 
- executemany(sql): insert, update, delete 배치 처리
    - insert, update, delete는 다 처리후 conn.commit()으로 커밋 처리해야한다.
    

#### select 결과 조회 메소드
- execute()로 실행한 SQL이 select인 경우 다음 메소드로 조회한다.
- fetchall()  : 조회된 모든 행을 한번에 가져올 때 사용한다. 결과를 tuple들을 묶은 리스트로 반환
- fetchone()  : 호출시 마다 한행씩 반환한다. PK로 조회한 경우 많이 사용한다.
- fetchmany(n): n행만큼 조회한다. n기본값-100. 특정개수반큼 반복문을 이용해 가져올때 사용.

In [8]:
import cx_Oracle
from pprint import pprint  #자료구조를 보기 좋게 출력해주는 print함수

In [16]:
# sql문: 문자열로 작성, 대소문자 구별 X (sql의 keyword들은 보통 대문자, 컬럼/테이블명은 소문자)
#        세미콜론은 붙이지 않음
select_sql = "SELECT emp_id, emp_name, job_id FROM emp"
insert_sql = "INSERT INTO emp VALUES (1500, '홍길동', 'FI_ACCOUNT', 101, to_date('2021/02/15', 'yyyy/mm/dd'), 30000, 0.2, 100)"
conn = None  #Connection을 대입할 변수
cursor = None  #Cursor 객체를 대입할 변수

try:
    #1. 연결
    conn = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
    print("연결 완료")
    #2. Cursor 생성
    cursor = conn.cursor()
    #3. sql 실행
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
    #4. select 결과 조회
    result = cursor.fetchall()  #조회 결과를 한 번에 반환. 한 행-Tuple, 리스트로 tuple들을 묶음
    print(type(result), type(result[0]))
    print(len(result))
    pprint(result)
    #5. insert/update/delete 후에 반드시 commit
    conn.commit()
except Exception as e:
    print(e)
finally:
    #6. 연결 닫기 - cursor/connection
    if conn != None:
        conn.close()
    if conn != None:
        conn.close()

연결 완료
<class 'list'> <class 'tuple'>
68
[(100, 'Steven', 'AD_PRES'),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (140, 'Joshua', None),
 (145, 'John', 'SA_MAN'),
 (146, 'Karen', 'SA_MAN'),
 (147, 'Alberto', 'SA_MAN'),
 (150, 'Peter', 'SA_REP'),
 (151, 'David', 'SA_REP'),
 (152, 'Peter', 'SA_REP'),
 (153, 'Christopher', 'SA_REP'),
 (154, 'Nanette', 'SA_REP'),
 (155, 'Oliver', 'SA_REP'),
 (156, 'Janette', 'SA_REP'),
 (15

## placeholder(자리표시자)를 이용해 sql 실행
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식
    - 리스트나 튜플: placeholder 순서

In [17]:
import cx_Oracle
from pprint import pprint

In [20]:
select_sql = "SELECT * FROM emp WHERE salary BETWEEN :1 and :2"
insert_sql = "INSERT INTO emp VALUES(:id, :name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"

conn = None
cursor = None

try:
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE')
    cursor = conn.cursor()
# SQL문 실행
# varchar2/char/nvarchar/nchar : str
# number (정수/실수) : int/double
    param_dict = {
        'id':2000,
        'name':'이순신',
        'job_id':'FI_ACCOUNT',
        'mgr_id':101,
        'hire_date':'20210213',
        'salary':19990,
        'dept_id':110
    }
    
    cursor.execute(select_sql, [10000, 20000])
    result1 = cursor.fetchall()
    print("15000 ~ 20000 사이 salary 조회")
    pprint(result)
    
    cursor.execute(select_sql, [2000, 3000])
    result2 = cursor.fetchall()
    print("2000 ~ 3000 사이 salary 조회")
    pprint(result2)
    
except Exception as e:
    print(e)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        
    

15000 ~ 20000 사이 salary 조회
[(100, 'Steven', 'AD_PRES'),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (140, 'Joshua', None),
 (145, 'John', 'SA_MAN'),
 (146, 'Karen', 'SA_MAN'),
 (147, 'Alberto', 'SA_MAN'),
 (150, 'Peter', 'SA_REP'),
 (151, 'David', 'SA_REP'),
 (152, 'Peter', 'SA_REP'),
 (153, 'Christopher', 'SA_REP'),
 (154, 'Nanette', 'SA_REP'),
 (155, 'Oliver', 'SA_REP'),
 (156, 'Janette', 'SA_REP'),
 (157, 'Patrick',

## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.
- 반환 type: tuple

### fetchall()
- 조회한 전체 결과셋을 반환한다.
- 한 row를 Tuple로 묶고 그 Tuple들을 List로 묶어서 반환한다.

### fetchmany(n)
- n 행씩 조회할 때 사용
- n 기본값 100

In [21]:
# emp_id(PK)로 직원 정보를 조회하는 함수
def select_emp_by_id(emp_id):
    sql = "SELECT * FROM emp WHERE emp_id = :1"
#     conn, cursor = None, None
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            #sql 실행
            cursor.execute(sql, [emp_id])
            result = cursor.fetchone()  #1행 조회. 조회결과 없으면 None 반환
            return result
    

In [22]:
r = select_emp_by_id(100)
print(r, "id:", r[0], '이름:', r[1])

(100, 'Steven', 'AD_PRES', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 24000.0, None, 90) id: 100 이름: Steven


In [23]:
r = select_emp_by_id(3000)
print(r)

None


In [25]:
sql = 'SELECT * FROM emp'
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(10)  #10개만 조회
        print(len(result))
        print(result)
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)

10
[(100, 'Steven', 'AD_PRES', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 24000.0, None, 90), (101, 'Neena', 'AD_VP', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 17000.0, None, 90), (102, 'Lex', 'AD_VP', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 17000.0, None, 90), (103, 'Alexander', 'IT_PROG', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 27000.0, None, 60), (104, 'Bruce', 'IT_PROG', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 18000.0, None, 60), (105, 'David', 'IT_PROG', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 14400.0, None, 60), (106, 'Valli', 'IT_PROG', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 14400.0, None, 60), (107, 'Diana', 'IT_PROG', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 12600.0, None, 60), (108, 'Nancy', 'FI_MGR', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 12008.0, None, 100), (109, 'Daniel', 'FI_ACCOUNT', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 9000.0, None, 100)]
10
[(110, 'John', 'FI_ACCOUNT', None, datet

### cursor를 for in 문을 이용해 select결과 조회
- select후 cursor는 Iterable 한 객체.
- cursor는 반복시마다 조회한 한행을 반환.

In [27]:
sql = "SELECT * FROM emp WHERE comm_pct IS NOT NULL ORDER BY emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        for row in cursor:  #한 번 반복할 떄마다 cursorfetchone() 실행
            print(row)

In [28]:
sql = "SELECT job_id, sum(salary), round(avg(salary), 2) FROM emp GROUP BY job_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)


[('AD_VP', 34000, 17000),
 ('FI_ACCOUNT', 43050, 8610),
 ('HR_REP', 9750, 9750),
 ('PU_MAN', 20100, 10050),
 ('AC_MGR', 12008, 12008),
 ('AD_ASST', 6600, 6600),
 ('IT_PROG', 86400, 17280),
 (None, 24300, 4050),
 ('SA_MAN', 39500, 13166.67),
 ('AC_ACCOUNT', 8300, 8300),
 ('FI_MGR', 12008, 12008),
 ('ST_MAN', 42550, 8510),
 ('AD_PRES', 24000, 24000),
 ('MK_MAN', 13000, 13000),
 ('SA_REP', 276850, 9228.33),
 ('MK_REP', 9000, 9000),
 ('PR_REP', 10000, 10000)]


In [29]:
sql = "SELECT e.emp_name, d.dept_name FROM emp e left join dept d ON e.dept_id = d.dept_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)


[('Jennifer', 'Administration'),
 ('Michael', 'Marketing'),
 ('Pat', 'Marketing'),
 ('Den', 'Purchasing'),
 ('Alexander', 'Purchasing'),
 ('Susan', 'Human Resources'),
 ('Matthew', 'Shipping'),
 ('Adam', 'Shipping'),
 ('Payam', 'Shipping'),
 ('Shanta', 'Shipping'),
 ('Kevin', 'Shipping'),
 ('Mozhe', 'Shipping'),
 ('James', 'Shipping'),
 ('Joshua', 'Shipping'),
 ('Anthony', 'Shipping'),
 ('Timothy', 'Shipping'),
 ('Randall', 'Shipping'),
 ('Alexander', 'IT'),
 ('Bruce', 'IT'),
 ('David', 'IT'),
 ('Valli', 'IT'),
 ('Diana', 'IT'),
 ('Hermann', 'Public Relations'),
 ('John', 'Sales'),
 ('Karen', 'Sales'),
 ('Alberto', 'Sales'),
 ('Peter', 'Sales'),
 ('David', 'Sales'),
 ('Peter', 'Sales'),
 ('Christopher', 'Sales'),
 ('Nanette', 'Sales'),
 ('Oliver', 'Sales'),
 ('Janette', 'Sales'),
 ('Patrick', 'Sales'),
 ('Allan', 'Sales'),
 ('Lindsey', 'Sales'),
 ('Louise', 'Sales'),
 ('Sarath', 'Sales'),
 ('Clara', 'Sales'),
 ('Danielle', 'Sales'),
 ('Mattea', 'Sales'),
 ('David', 'Sales'),
 ('Sundar'


## INSERT / UPDATE/ DELETE
- SQL 실행 후 Connection객체의 commit()을 호출 해야 최종적으로 적용된다.
- commit()
    - 최종 적용
- rollback()
    - 변경전 상태로 되돌리기

### 사용할 테이블
```sql
create table test(
    id varchar2(10),
    name varchar2(100),
    age number(3));
```

In [30]:
create_sql = 'create table test(id varchar2(10), name varchar2(100), age number(3))'
insert_sql = 'INSERT INTO test VALUES (:1, :2, :3)'

with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        try:
            cursor.execute(create_sql)  #테이블 생성
        except Exception as e:
            print(e)
        #insert
        cursor.execute(insert_sql, [1, '이름1', 20])
        cursor.execute(insert_sql, [2, '이름2', 30])
        cursor.execute(insert_sql, [3, '이름3', 40])
        conn.commit()  #commit
        

In [38]:
update_sql = 'UPDATE test SET name=:!'
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(update_sql, ['새이름'])
        conn.commit()

DatabaseError: ORA-01036: 잘못된 변수명/번호

In [35]:
delete_sql = 'DELETE FROM test WHERE name = :1'
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(delete_sql, ['이름1'])
        conn.commit()

In [39]:
select_sql = "select * from test"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(select_sql)
        pprint(cursor.fetchall())

[('2', '이름2', 30), ('3', '이름3', 40)]


## batch(일괄작업) 처리
- 한번에 메소드 호출로 다수 행을 처리한다.
- executemany(sql, placeholder에 전달할값)
    - placeholder에 전달할 값을 list로 묶어서 전달하면 sql문이 그 개수만큼 한번에 처리된다.

In [ ]:
insert_sql = 'INSERT INTO test VALUES (:1, :2, :3)'
param = [
    ['a-1', '이름1', 20],
    ['a-2', '이름2', 30],
    ['a-3', '이름3', 35],
    ['a-4', '이름4', 40]
]
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql, param)

## TODO : 매개변수로 dept_id를 받아서 EMP 테이블에서 그 부서의 직원들의 salary를 두배 update하는 함수.

In [1]:
import cx_Oracle
def update_salary_by_dept_id(dept_id):
    sql = "UPDATE emp SET salary = salary * 2 WHERE dept_id = :1"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:  #커서 생성
            cursor.execute(sql, [dept_id])  #update문 실해
            conn.commit()

In [ ]:
update_salary_by_dept_id(80)

## TODO : 매개변수로 job_id를 받아서 그 업무를 하는 직원들을 EMP 테이블에서 삭제하는 함수.

In [5]:
def delete_emp_by_job_id(job_id):
    sql = 'DELETE FROM emp WHERE job_id = :1'
    with cx_Oracle.connect('c##scott_join', 'tiger', 'localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [job_id])
            conn.commit()

In [ ]:
# job_id가 null인데 지우고 싶다면?
def delete_emp_by_job_id(job_id):
    if job_id != None:
        sql = 'DELETE FROM emp WHERE job_id = :1'
    else:
        sql = 'DELETE FROM emp WHERE job_id is null'
        
    with cx_Oracle.connect('c##scott_join', 'tiger', 'localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            if job_id != None:
                cursor.execute(sql, [job_id])
            else:
                cursor.execute(sql)
            conn.commit()

In [6]:
delete_emp_by_job_id('AD_VP')

# TODO : 매개변수로 직원 이름을 입력받아 EMP 테이블에서 그  직원들의 전체 정보를 조회결과를 출력하는 함수.

In [14]:
import cx_Oracle
def select_emp_by_name(emp_name):
#     sql = "SELECT * FROM emp WHERE emp_name = :1"
    sql = "SELECT * FROM emp WHERE emp_name LIKE '%'||:name||'%'"  #Like 연산자에 placeholder 쓸 때 이렇게 써야함
    with cx_Oracle.connect('c##scott_join', 'tiger', 'localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
#             cursor.execute(sql, [emp_name])
            
            result = cursor.fetchall()
            return result

In [13]:
r = select_emp_by_name('John')
print(r)

[(110, 'John', 'FI_ACCOUNT', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 8200.0, None, 100), (145, 'John', 'SA_MAN', None, datetime.datetime(2021, 2, 5, 12, 14, 31), 14000.0, None, 80)]


# TODO: 사용자로 부터 직원정보를 입력받아 EMP 테이블에 입력한다.
- emp_id, emp_name, job_id, mgr_id, hire_date, salary, comm_pct, dept_id 를 하나씩 입력받는다.
- job_id, mgr_id, salary, comm_pct, dept_id 을 입력하지 않은 경우 NULL 을 넣는다. (placehold 사용시 None을 대입)
- 입력된 값을 EMP 테이블에 insert 한다.
- 주의 job_id, mgr_id, dept_id는 FK 컬럼이므로 부모테이블의 PK값을 넣어야 한다.

In [20]:
def insert_emp():
    emp_id = int(input("직원ID: "))
    emp_name = input("이름: ")
    job_id = input("업무ID: ")
    if len(job_id.strip()) == 0:
        job_id = None
        
    try:
        mgr_id = int(input("상사ID: "))
    except:
        mgr_id = None
        
    hire_date = input("입사일(형식: yyyymmdd): ")
    salary = int(input("급여: "))
    
    try:
        comm_pct = float(input("커미션율: "))
        if comm_pct >= 1:
            raise Exception()
    except:
        comm_pct = None
        
    try:
        dept_id = int(input("부서ID: "))
    except:
        dept_id = None
    
    param = {
        "emp_id":emp_id,
        "emp_name":emp_name,
        "job_id":job_id,
        "mgr_id":mgr_id,
        "hire_date":hire_date,
        "salary":salary,
        "comm_pct":comm_pct,
        "dept_id":dept_id
    }
    sql = INSERT INTO emp VALUES (:emp_id, :emp_name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)
    with cx_Oracle.connect("c##scott_join", "tiger", "localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, param)
            conn.commit()
            print("insert 완료")

SyntaxError: invalid syntax (<ipython-input-20-a3af13168ce8>, line 38)